In [244]:
import pandas as pd
#import matplotlib.pyplot as plt
#import descartes
import geopandas as gpd #cond install geopandas
#from shapely.geometry import Point, Polygon
import folium
import numpy as np


%matplotlib inline
#https://towardsdatascience.com/geopandas-101-plot-any-data-with-a-latitude-and-longitude-on-a-map-98e01944b972

In [245]:
df = pd.read_csv('Tripadvisordata.csv')
df[["Latitude","Longitude"]] = df['Location'].str.split(',',expand=True)
crs = {'init':'epsg:4326'}
df["Longitude"] = df["Longitude"].astype(float)
df["Latitude"] = df["Latitude"].astype(float)
df["Ranking on list"] = df["Ranking on list"].astype(int)
df["Color"] = ""
df["Color_price] = ""

In [246]:
df.head()

,Name,Main rating,Ranking on list,Price range,Price class,Number of reviews,Location,Distance from Kgs. Nytorv,Service,Good price,Food,Atmosphere,Latitude,Longitude,Color
0,Kokkeriet,4.5,13,------16770.-970.,--$$$$---,769 anmeldelser,"55.687626,12.585126",853.0,5.0,4.5,5.0,NaN,55.687626,12.585126,
1,The Olive Kitchen & Bar,5.0,3,------1674.-174.,--$$-$$$---------,2.138 anmeldelser,"55.68125,12.571284",821.0,5.0,4.5,5.0,NaN,55.681250,12.571284,
2,Pizzeria Mamemi & Wine Bar,5.0,7,"------16,,,",--$---------,927 anmeldelser,"55.667767,12.548116",2638.0,5.0,4.5,5.0,NaN,55.667767,12.548116,
3,Zahida at Kellerdirk,5.0,5,------1647.-248.,--$$-$$$---------,878 anmeldelser,"55.67162,12.546196",2557.0,5.0,4.5,5.0,NaN,55.671620,12.546196,
4,Grams Lækkerier,5.0,1,------1633.-100.,--$---------,720 anmeldelser,"55.671036,12.562724",1673.0,5.0,4.5,5.0,4.5,55.671036,12.562724,


In [251]:
def color_producer(x):
    if x < 200:
        return '#00FF00' #green
    elif x > 200 and x < 300:
        return '#9FFF00' #light green
    elif x > 400 and x < 500:
        return '#F7FF00' #yellow
    elif x > 600 and x < 700:
        return '#FFAF00' #light orange 
    elif x > 800 and x < 900:
        return '#FF5700' #dark orange
    elif x > 900 and x < 1000:
        return '#FF0000' #red
    else:
        return 'black'

for x in df["Ranking on list"].index:
        df["Color"][x] = color_producer(df["Ranking on list"][x])

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [282]:
map_rest = folium.Map(location = [55.679977,12.5841893], zoom_start = 12)  #tiles='Stamen Toner'

In [283]:
for x,y,z,v in zip(df["Latitude"],df["Longitude"],df["Color"],df["Name"]):
    folium.CircleMarker([x,y],radius=0.5, color = z, tooltip = v).add_to(map_rest)

In [284]:
map_rest

In [ ]:
def color_price_class(x):
    
    dollar_dict = {
    '$$$$ - $$$$$' : '4.5',
    '$$$ - $$$$' : '3.5',
    '$$ - $$$' : '2.5',
    '$ - $' : '1.5',
    
    '$$$$$' : '5',
    '$$$$' : '4',
    '$$$' : '3',
    '$$' : '2',
    '$' : '1'
    }
    
    
    if x == "$" or x == "$ - $":
        return '#00FF00' #green
    elif x == "$$" or x == "$$ - $$$":
        return '#9FFF00' #light green
    elif x == "$$$" or x == "$$$ - $$$$":
        return '#F7FF00' #yellow
    elif x == "$$$$" or x == "$$$$ - $$$$$":
        return '#FFAF00' #light orange 
    elif x == "$$$$$"
        return '#FF0000' #red
    else:
        return 'black'

    
for x in df["price"].index:
        df["Color_price"][x] = color_producer(df["Ranking on list"][x])